In [ ]:
import logging

import numpy as np
import matplotlib.pyplot as plt

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s %(levelname)s %(name)-40s :: %(message)s'
)


# Phaseless Inverse Obstacle Scattering Problem 

Operator that maps the shape of a sound-soft obstacle to the phaseless far-field measurements $(|u_{\infty}(\hat{x},\hat{d})|^2)$. 
The scattering problem is described by

$$
        \begin{cases}
            \Delta u +\kappa^2 u = 0 & \text{ in } \mathbb{R}^2\backslash\overline{D}\\
             u = 0  & \text{ on } \partial D\\
            \displaystyle{\lim_{r\to\infty}}r^{\frac{1}{2}}(\frac{\partial u^s}{\partial r}-i\kappa u^s)=0 & \text{ for } r=|x|,
        \end{cases}
$$
where $u=u^s+u^i$ is the total field and $D$ is a bounded obstacle in $\mathbb{R}^2$ with $\partial D\in\mathcal{C}^2$.
    

In [ ]:

from dirichlet_op import DirichletOp
from regpy.operators import SquaredModulus

#Forward operator
op_dir = DirichletOp(
    kappa = 3,
    N_inc=[np.array([1,0]), np.array([0,1]), np.array([-1, 0]), np.array([0,-1])]
)
op_sqm = SquaredModulus(op_dir.codomain)

op = op_sqm * op_dir


In [ ]:
from regpy.solvers import RegularizationSetting
from regpy.hilbert import L2, Sobolev
from dirichlet_op import create_synthetic_data
from regpy.vecsps.curve import apple

setting = RegularizationSetting(op=op, penalty=Sobolev, data_fid=L2)

N_ieq_synth=64
#Exact data
farfield, exact_solution = create_synthetic_data(op_dir, true_curve=apple(N_ieq_synth,der=3))

phaseless_farfield = op_sqm(farfield)

# Poisson data
N = 2000
data = np.sum(np.random.poisson(phaseless_farfield, (N,) + op.codomain.shape), axis=0)
data = data/N

noiselevel = setting.h_codomain.norm(data-phaseless_farfield)
print("Noise level=", noiselevel)

#Initial guess
t = 2*np.pi*np.arange(0, op_dir.N_FK)/op_dir.N_FK
init = 0.45*np.append(np.cos(t), np.sin(t)).reshape((2, op_dir.N_FK))
init=init.flatten()

In [ ]:
x= op_sqm.codomain.coords[0]
y = phaseless_farfield[:,0]
plt.plot(x,np.real(y))


In [ ]:
from regpy.solvers.nonlinear.irgnm import IrgnmCG
from regpy.solvers.nonlinear.newton import NewtonCG
import regpy.stoprules as rules

#Solver: NewtonCG or IrgnmCG
solver = NewtonCG(
    setting, data, init = init,
        cgmaxit=50, rho=0.5
)

"""
solver = IrgnmCG(
    setting, data,
    regpar=1.,
    regpar_step=0.5,
    init=init
)
"""
stoprule = (
    rules.CountIterations(100) +
    rules.Discrepancy(
        setting.h_codomain.norm, data,
        noiselevel=noiselevel,
        tau= 1.5
    )
)

#Plot function
fig, axs = plt.subplots(1, 2)
axs[0].set_title('Obstacle')
axs[1].set_title('Phaseless Farfield')

reco, reco_data = solver.run(stoprule)
axs[0].plot(*exact_solution.z)
axs[0].plot(*op_dir.domain.bd_eval(reco, nvals=op_dir.N_ieq, nderivs=3).z)

axs[1].plot(op.codomain.coords[0][:,0], phaseless_farfield.real[:,0], label='exact')
axs[1].plot(op.codomain.coords[0][:,0], reco_data.real[:,0], label='reco')
axs[1].plot(op.codomain.coords[0][:,0], data.real[:,0], label='measured')
axs[1].legend()
plt.show()